<h4>This dataset presents a list of the 100 most streamed songs on Spotify of all time</h4>
(This analysis dates from March 8th 2023)  


Dataset source: https://www.kaggle.com/datasets/amaanansari09/most-streamed-songs-all-time




<h4>Features description:</h4>

- <b>duration:</b> Duration of the song (minutes).


- <b>energy:</b> A perceptual measure of intensity and activity.


- <b>key:</b> The harmony key, being 0 = C, 1 = C#, ... , 11 = B


- <b>loudness:</b> The overall loudness of a track in decibels (dB).


- <b>mode:</b> 0 = Major Key; 1 = Minor Key;


- <b>speechiness:</b> ?


- <b>acousticness:</b> ?


- <b>instrumentalness:</b> ?


- <b>liveness:</b> ?


- <b>valence:</b> ?


- <b>tempo:</b> Rhythm of the song (beats/min or bpm)


- <b>danceability:</b> ?






<h3>Objectives and Key Insights</h3>

<h4> Answers to be found:</h4>

1. How many artists made it to the top 100 most streamed songs of all time on Spotify?

2. Who are the artists with more songs in the list

3. Who are the artists with more streams in the list

4. Is there a pattern in terms of "key" and "mode" for these top songs?

5. Do these songs gravitate around an "optimal duration"?

6. Is there a correlation between "energy" and "tempo"?

7. Do these songs gravitate around an "optimal duration"?

8. What are the songs with more streams per day?



<h4> Insights that cannot be leveraged with this dataset (and I am particularly interested in!!):</h4>

1. What are the chord progressions of each song? Is there a pattern among these top streamed ones?

2. What is the primary language the song is sung in? And how many words are there?

3. What are the song structures used by each song? (e.g. Intro, Verse, Bridge, Pre Chorus, Chorus, Otro, etc.)

4. What is the total number of streams these artists have now and before launching each of these songs?


In [37]:
# Import everything you need
import os
import pandas as pd
import numpy as np 
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date
from dateutil.parser import parse


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 10000)

In [38]:
# Set-up the input files with da data to be analyzed
features_file_path = f"/Users/sandrolobao/Desktop/Python Projects/Streamed Songs Spotify/streamed songs spotify/Features.csv"

streams_file_path = f"/Users/sandrolobao/Desktop/Python Projects/Streamed Songs Spotify/streamed songs spotify/Streams.csv"

In [39]:
# Do all transformations needed to your data and enable analysis
df_features = pd.read_csv(features_file_path)
df_streams = pd.read_csv(streams_file_path)

df_merge = pd.merge(left=df_streams, left_on="Song", right=df_features, right_on="name")

df_merge = df_merge[["Song", "Artist", "Streams (Billions)", "Release Date", "duration", "energy", "key", "mode", "loudness", "tempo"]]

todays_date = date(2023, 3, 8)
df_merge.insert(4, "Today", todays_date)
df_merge['Release Date'] = df_merge['Release Date'].astype('string')
df_merge['Release Date'] = df_merge['Release Date'].apply(lambda x: parse(x).date())

df_merge['Aging in Days'] = df_merge['Today'] - df_merge['Release Date']
df_merge['Aging in Days'] = df_merge['Aging in Days'].dt.total_seconds() / (60*60*24)

df_merge.head()

,Song,Artist,Streams (Billions),Release Date,Today,duration,energy,key,mode,loudness,tempo,Aging in Days
0,Blinding Lights,The Weeknd,3.449,2019-11-29,2023-03-08,3.33,0.730,1,1,-5.934,171.005,1195.0
1,Shape of You,Ed Sheeran,3.398,2017-01-06,2023-03-08,3.90,0.652,1,0,-3.183,95.977,2252.0
2,Dance Monkey,Tones And I,2.770,2019-05-10,2023-03-08,3.49,0.588,6,0,-6.400,98.027,1398.0
3,Someone You Loved,Lewis Capaldi,2.680,2018-11-08,2023-03-08,3.04,0.405,1,1,-5.679,109.891,1581.0
4,Rockstar,Post Malone featuring 21 Savage,2.620,2017-09-15,2023-03-08,3.64,0.520,5,0,-6.136,159.801,2000.0


TODO: Address the issue of the "featuring" artists (e.g. "Post Malone featuring 21 Savage" is a different singer when compared to "Post Malone"). Perhaps I should consider only the main singer...

TODO: I would like to see what does the average streams per day look like for them...

In [40]:
# Function to split "featuring" and "and" artists and concat
def splitArtist(dataframe, separator):
    aux = dataframe[dataframe['Artist'].str.contains(r'\b\s{0}\s\b'.format(separator), case=True)]
    aux = aux['Artist'].str.split(pat=r'\s{0}\s'.format(separator), n=1, expand=True)
    aux = pd.concat([aux[0], aux[1]], axis=0)
    return aux

In [41]:
# Generate all artists list
def findAllArtists(artist_objs_list, separator_word_list, sep):
    aux = pd.concat(artist_objs_list, axis=0)
    
    # This could be done in a single line, but this way has better readability
    regex_aux = r'\b\s{0}\s\b'
    sep_aux = sep.join(separator_word_list)
    filter_aux = aux.str.contains(regex_aux.format(sep_aux))
    aux = aux[~filter_aux]
    # Alternative:
    # aux = aux[~aux.str.contains(r'\b\s({0})\s\b'.format(sep.join(separator_word_list)))]
    
    aux = aux.dropna()
    aux = aux.drop_duplicates()
    return aux

In [42]:
feat_df = splitArtist(df_merge, 'featuring')
and_df = splitArtist(df_merge, 'and')

artist_objs_list = [feat_df, and_df, df_merge['Artist']]
separator_word_list = ['featuring', 'and']
separator = '|'

all_artists = findAllArtists(artist_objs_list=artist_objs_list,         separator_word_list=separator_word_list, sep=separator)
len(all_artists)

86